In [1]:
import torch
import sys
import numpy as np
from torch import nn
sys.path.append("../")
from MLRF.datatools import h5kit
from scipy.signal import find_peaks


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv1d(
            in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1
        )
        self.conv2 = nn.Conv1d(
            in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1
        )
        self.conv3 = nn.Conv1d(
            in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1
        )
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 256, 128)
        self.dropout = nn.Dropout(p=0.1)
        self.fc2 = nn.Linear(128, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        if len(x.shape) == 2:
            x = x.unsqueeze(1)

        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))

        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        # x = self.dropout(x)
        x = self.fc2(x)

        return x

In [2]:
# Load the model

model = torch.load('model.pth', map_location=device)
model.to(device)
model.eval()

# Example data for inference
dataset = h5kit("../overair.h5")

keys = dataset.keys()
NOISE_THRESHOLD = -1000
PEAK_PROMINENCE = 2
# Perform inference
with torch.no_grad():
    wifi_count = 0
    bluetooth_count = 0

    for key in keys:
        data = dataset.read(key).astype(np.float32)
        peaks, properties = find_peaks(
        data, height=NOISE_THRESHOLD, prominence=PEAK_PROMINENCE
        )
        if len(peaks) > 0:
            data = torch.tensor(data).unsqueeze(0).unsqueeze(0).to(device)
            output = model(data)
            pred = torch.argmax(output).item()
            if pred == 0:
                wifi_count += 1
                decision = "wifi"
            else:
                bluetooth_count += 1
                decision = "bluetooth"

    print(f"Total wifi: {wifi_count}, Total bluetooth: {bluetooth_count}")

/tmp/ipykernel_1056/2625785872.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('model.pth', map_location=device)


Total wifi: 0, Total bluetooth: 868
